# Joint cascade model with $A_\rm{eff}(E, \omega)$

Add in spline evaluation to deal with 2D $A_\rm{eff}$. Use simple example from the spline development examples.

In [1]:
import numpy as np
import h5py
from matplotlib import pyplot as plt
import stan_utility
import corner
from joint_model_cascades import * 

/Users/fran/projects/bayes/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
random_seed = 19920715

## Forward model

In [3]:
# Use a set of sources uniform in comoving volume for a lambda-CDM universe.
with h5py.File('data/test_SFR_pop.h5', 'r') as f:
    Ns = f['output/Ns'].value
    redshift = f['output/redshift'].value
    position = f['output/position'].value
    
# Select sources within redshift zth to mimic catalogue
zth = 1
selection = np.where(redshift < zth)[0]

In [4]:
Ns = len(selection)
redshift = np.array([redshift[_] for _ in selection])
position = np.array([position[_] for _ in selection])

# Try ordering sources by redshift, as larger z => lower F
# This could help eliminate degeneracies in the mixture
inds = redshift.argsort()[::-1]
redshift = redshift[inds]
position = position[inds]
unit_vector = [p / np.linalg.norm(p) for p in position]

# Add background redshift
redshift = list(redshift) + [zth]

In [ ]:
# Use simple Aeff calculated in `spline_dev/clean_spline_snippet.ipynb`
# Define spline
p = 3 # spline degree
tx_orig = np.array([2.90998783, 4.03010253, 5.00991847, 6.96995941]) # knot sequence
ty_orig = np.array([-0.9,  0.1,  0.9]) # knot sequence
Nx = len(tx_orig)+p-1 # number of coefficients that need to be defined
Ny = len(ty_orig)+p-1 # number of coefficients that need to be defined
N = Nx * Ny

# Coefficients
c = np.asarray(
    [-3.58077707, -3.41361407, -3.67857551, -4.18870088, -4.33580191, -1.96090326,
     -1.97689,    -2.05099873, -2.14549395, -2.17675871, -0.34077015, -0.32737177,
     -0.30599357, -0.29709922, -0.29278352,  0.48041494,  0.94140028,  1.02319478,
     0.88547053,  0.95448815, -0.26299716,  0.99602738,  1.47548988,  1.31308221,
     1.42759428, -1.33184261,  0.47994404,  1.74747881,  1.44319947,  1.39107924])
c = c.reshape(Nx, Ny)

In [6]:
np.log10(10)

1.0